In [14]:
!nvidia-smi
!pip install roboflow -q
!pip install ultralytics==8.0.196 -q
!sudo apt-get install unzip -q

Sat Mar  9 01:07:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import os
from datetime import date
from roboflow import Roboflow
import ultralytics
from google.colab import drive
import os
import shutil

WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'.


## Utilities

In [ ]:
def mkdir_folder(folder, path):
  PATH = os.path.join(path,folder)
  if os.path.exists(PATH) == False:
    os.mkdir(PATH)
    print(f"Folder'{folder}' was created on path {path}")
  else:
    print(f"Folder '{folder}' was previously created on path '{path}'")

  return

def copy_folder_contents(source_folder, destination_folder):
    # Create destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        return f"WARNING: Folder {destination_folder} doesn't exist!!!"

    # Iterate over files in the source folder
    for item in os.listdir(source_folder):
        source_item = os.path.join(source_folder, item)
        destination_item = os.path.join(destination_folder, item)

        # Check if the item is a file
        if os.path.isfile(source_item):
            # Copy the file to the destination folder
            shutil.copy2(source_item, destination_item)
        elif os.path.isdir(source_item):
            # Recursively copy subfolders
            shutil.copytree(source_item, destination_item)

## Dataset Definition

In [6]:
''' --------------- PARAMETROS DEL DATASET  ---------------------'''
DATASET_NAME = 'blueberry2'
DATASET_VERSION = 7
MODEL = 'yolov8'
'''-----------------------------------------------------------'''

# downloading the dataset
rf = Roboflow(api_key="Z63mUXBjG48P7Hkb3vDi")
project = rf.workspace("pqbas").project(DATASET_NAME)
dataset = project.version(DATASET_VERSION).download(MODEL)


## Training

In [12]:
''' --------------- PARAMETROS DE ENTRENAMIENTO -----------------'''
EPOCHS = 2
BATCH = 24
VERSION = 'yolov8m'
'''-----------------------------------------------------------'''

# training with yolov8
!yolo train model=yolov8m.pt data={dataset.location}/data.yaml batch={BATCH} epochs={EPOCHS} imgsz=640 name={VERSION} device=0

New https://pypi.org/project/ultralytics/8.1.24 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/Blueberry2-7/data.yaml, epochs=2, patience=50, batch=24, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=yolov8m, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False,

## Save results

In [15]:
HOME = os.getcwd()
today = str(date.today())
PATH = f'{HOME}/drive/MyDrive'

# create the directory were experiments will be saved
mkdir_folder(today, PATH)

# copy experiments to folder
copy_folder_contents(os.path.join(HOME,'runs','detect'), os.path.join(PATH,today))

# Things that I don't understand

In [ ]:
!rm -r /content/runs
!rm -r /content/detection
!cp -r /content/runs/detect/yolov8m {PATH}

DATASET_PATH = os.path.join(HOME, DATASET_NAME.capitalize() + "-" + str(DATASET_VERSION))

# create the directory where the datasets will be saved
mkdir_folder('datasets', HOME)

# move the dataset downloaded to datasets folder
copy_folder_contents(DATASET_PATH, os.path.join(HOME,'datasets'))